### Campaign profit - Sensitivity analysis

In [1]:
#
##
### LITTLE HELPERS

import pandas as pd
import numpy as np
from itertools import product

def _optimize_numeric_dtypes(df):
    import pandas as pd
    float_cols = df.select_dtypes("float").columns
    int_cols = df.select_dtypes("integer").columns
    df[float_cols] = df[float_cols].\
        apply(pd.to_numeric, downcast="float")
    df[int_cols] = df[int_cols].\
        apply(pd.to_numeric, downcast="integer")
    return df

def get_data(path="../data/retailrocket/raw/customer_model/"):

    data = _optimize_numeric_dtypes(pd.read_parquet(path))
    cf = ["user_id","target_event", "target_customer_value_lag1", "target_customer_value"]
    data = data.loc[:,cf]
    return data

def get_acp_stats(data, config):
    gamma = config["gamma"]
    delta = config["delta"]
    psi = config["psi"]
    n_iter = config["n_iter"]
    np.random.seed(config["seed"])

    n_users = data.user_id.nunique()
    sp = []
    for i in range(n_iter):
        gamma_psi = pd.DataFrame.from_dict({
            "user_id":data.user_id.unique(),
            "gamma":np.random.beta(gamma["alpha"], gamma["beta"], size=n_users),
            "psi":np.random.beta(psi["alpha"], psi["beta"], size=n_users)})
        temp = data.merge(gamma_psi, on=["user_id"])
        #temp["ecp"] = (temp["y_pred_proba"]*temp["gamma"]*(temp["target_customer_value_lag1"]-delta)
        #    + (1-temp["y_pred_proba"])*(-temp["psi"]*delta))
        temp["acp"] = (temp["target_event"]*temp["gamma"]*(temp["target_customer_value"]-delta)
            + (1-temp["target_event"])*(-temp["psi"]*delta))
        #sp.append(temp.sort_values("acp", ascending=False).acp.cumsum().max())
        sp.append(temp["acp"].mean())
    res = pd.Series(sp).describe(percentiles=[0.025, 0.975])[["mean", "2.5%", "97.5%"]]
    res["ga"] = gamma["alpha"]
    res["gb"] = gamma["beta"]
    res["gmu"] = gamma["mu"]
    res["gvar"] = gamma["var"]
    res["pa"] = psi["alpha"]
    res["pb"] = psi["beta"]
    res["delta"] = delta
    return res

def get_alpha_beta(mu, var):
    alpha = (-mu**3+mu**2-mu*var)/var
    beta = alpha*(1-mu)/mu
    return (alpha, beta)

def adjust_config(config, mu, var):
    config = config.copy()
    alpha, beta = get_alpha_beta(mu=mu,
        var=var)
    config["gamma"] = {"alpha":alpha, "beta":beta,
        "mu":mu, "var":var}
    return config

In [2]:
#
##
### ARE WE LIVIN IN A SIM?

mu = [0.001, 0.01, 0.1]
ln_var = [-7.645, -9.939, -12.921]

config_rr = {
    "gamma":{"alpha":2.042, "beta":202.116},
    "delta":7000, 
    "psi":{"alpha":6.12, "beta":3.15},
    "n_iter":1000,
    "seed":1}   

config_rees46 = {
    "gamma":{"alpha":2.042, "beta":202.116},
    "delta":20, 
    "psi":{"alpha":6.12, "beta":3.15},
    "n_iter":1000,
    "seed":1}       

configs = [adjust_config(config_rees46, mu, var)
    for mu, var in product(mu, np.exp(ln_var))]

data = get_data("../data/rees46/raw/customer_model/")
sim_stats = []
for con in configs:
    sim_stats.append(get_acp_stats(data, con))
sim_stats = pd.concat(sim_stats, axis=1).T
sim_stats["glnvar"] = np.log(sim_stats.gvar)    

In [3]:
sim_stats

,mean,2.5%,97.5%,ga,gb,gmu,gvar,pa,pb,delta,glnvar
0,-8.852746,-8.920856,-8.746909,0.001088,1.086990,0.001,0.000478,6.12,3.15,20.0,-7.645
1,-8.850635,-8.889610,-8.806941,0.019702,19.682586,0.001,0.000048,6.12,3.15,20.0,-9.939
2,-8.851658,-8.879951,-8.821947,0.407399,406.991431,0.001,0.000002,6.12,3.15,20.0,-12.921
3,-7.812083,-7.896480,-7.717599,0.196927,19.495742,0.010,0.000478,6.12,3.15,20.0,-7.645
4,-7.811142,-7.853554,-7.770582,2.041578,202.116230,0.010,0.000048,6.12,3.15,20.0,-9.939
5,-7.812215,-7.843563,-7.783131,40.461956,4005.733653,0.010,0.000002,6.12,3.15,20.0,-12.921
6,2.578651,2.486220,2.667594,18.711517,168.403655,0.100,0.000478,6.12,3.15,20.0,-7.645
7,2.582017,2.542509,2.620087,186.407098,1677.663884,0.100,0.000048,6.12,3.15,20.0,-9.939
8,2.582012,2.551541,2.613253,3679.168736,33112.518623,0.100,0.000002,6.12,3.15,20.0,-12.921
